# Experiment 01
## with visual interpretation
### Richardson et. al (2002) Experiment 01 with **Vision** Language Models instead of Humans

The subjects were presented with a single page,
containing a list of the verbs and four pictures, labelled A to
D. Each one contained a circle and a square aligned along a
vertical or horizontal axis, connected by an arrow pointing
up, down, left or right. Since we didn't expect any
interesting item variation between left or right placement of
the circle or square, the horizontal schemas differed only in
the direction of the arrow.
For each sentence, subjects were asked to select one of
the four sparse images that best depicted the event described
by the sentence (Figure 1)
The items were randomised in three different orders, and
crossed with two different orderings of the images. The six
lists were then distributed randomly to subjects.



### Setup of Experimental Environment

In [1]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
from transformers import AutoProcessor, AutoModelForVision2Seq
from huggingface_hub import hf_hub_download
import torch, random, requests
from PIL import Image

local_path = "/mounts/data/corp/huggingface/"
gpu_model_1 = "cuda:0"
gpu_model_2 = "cuda:1"
gpu_model_3 = "cuda:2"

def convert_to_float(value):
    try:
        return float(value)
    except ValueError:
        return value

/mounts/Users/cisintern/pwicke/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

### Loading preprocessed data by Richardson

Creates three dictionaries:
 * `richardson_data`
 
 All choices as vectors, e.g. `{'fled': [7.2, 4.2, 80.8, 7.8], 'pointed at': [7.2, 3.6, 0.0, 89.2] ...`
 
 * `richardson_categorial`
 
 Maximum choice as binary choice, e.g. `{'fled': [0, 0, 1, 0], 'pointed at': [0, 0, 0, 1] ...`
 
 * `richardson_normed`
 
 Maximum choice divided by all choices, disregarding all other choices, e.g.  `{'fled': [0.0, 0.0, 0.808, 0.0], 'pointed at': [0.0, 0.0, 0.0, 0.892] ...`
 

In [2]:
with open("../../data/richardson_actions.txt", "r") as d_in:
    lines = [line.split() for line in d_in.readlines()]

output = []
for entry in lines:
    new_entry = [convert_to_float(item) for item in entry]
    
    if isinstance(new_entry[1],str):
        new_entry[0] = " ".join(new_entry[:2])
        del new_entry[1]
    output.append(new_entry)

richardson_data = dict()
for elem in output:
    richardson_data[elem[0]] = [i for i in elem[1:]]

# Randomizing Richardson's data
action_words = list(richardson_data.keys())
random.shuffle(action_words)

richardson_categorial = dict()
for k, v in richardson_data.items():
    if k == 0:
        continue
    vals = [0,0,0,0]
    vals[v.index(max(v))] = 1

    richardson_categorial[k] = vals
richardson_normed = dict()

for action, values in richardson_data.items():
    if action == 0:
        continue
    
    richardson_normed[action] = [round(val/sum(values),4) for val in values]

print(richardson_normed)

{'fled': [0.072, 0.042, 0.808, 0.078], 'pointed at': [0.072, 0.036, 0.0, 0.892], 'pulled': [0.06, 0.054, 0.754, 0.132], 'pushed': [0.072, 0.036, 0.012, 0.88], 'walked': [0.0905, 0.0362, 0.2412, 0.6322], 'hunted': [0.0959, 0.2038, 0.018, 0.6823], 'impacted': [0.072, 0.371, 0.03, 0.527], 'perched': [0.12, 0.76, 0.066, 0.054], 'showed': [0.1499, 0.0899, 0.1019, 0.6583], 'smashed': [0.036, 0.665, 0.012, 0.287], 'bombed': [0.048, 0.868, 0.018, 0.066], 'flew': [0.377, 0.443, 0.15, 0.03], 'floated': [0.329, 0.563, 0.078, 0.03], 'lifted': [0.874, 0.096, 0.024, 0.006], 'sank': [0.2218, 0.7183, 0.042, 0.018], 'argued with': [0.1139, 0.1379, 0.1259, 0.6224], 'gave to': [0.084, 0.096, 0.012, 0.808], 'offended': [0.09, 0.317, 0.246, 0.347], 'rushed': [0.1025, 0.1085, 0.2352, 0.5538], 'warned': [0.1079, 0.2218, 0.0599, 0.6104], 'owned': [0.0539, 0.5564, 0.1858, 0.2038], 'regretted': [0.1978, 0.2398, 0.4126, 0.1499], 'rested': [0.144, 0.365, 0.401, 0.09], 'tempted': [0.168, 0.114, 0.455, 0.263], 'wan

## Choosing model

### Loading InstructBlip-Vicuna7b

In [3]:
model_instructblip = InstructBlipForConditionalGeneration.from_pretrained(local_path+"instructblip-vicuna/instructblip-vicuna-7b")
processor_instructblip = InstructBlipProcessor.from_pretrained(local_path+"instructblip-vicuna/instructblip-vicuna-7b")
model_instructblip.to(gpu_model_1);

Loading checkpoint shards: 100%|██████████| 4/4 [00:41<00:00, 10.27s/it]


In [3]:
# auto : tau 30sec to load 
model_instructblip = InstructBlipForConditionalGeneration.from_pretrained(local_path+"instructblip-vicuna/instructblip-vicuna-7b", device_map="auto")
processor_instructblip = InstructBlipProcessor.from_pretrained(local_path+"instructblip-vicuna/instructblip-vicuna-7b", device_map="auto")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 4/4 [00:23<00:00,  5.87s/it]


### Loading OpenFlamingo-9B-vitl

In [4]:
from open_flamingo import create_model_and_transforms
model_flamingo, image_processor, tokenizer = create_model_and_transforms(clip_vision_encoder_path="ViT-L-14",clip_vision_encoder_pretrained="openai",lang_encoder_path="anas-awadalla/mpt-7b",tokenizer_path="anas-awadalla/mpt-7b",cross_attn_every_n_layers=4)
checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-9B-vitl-mpt7b", "checkpoint.pt")
model_flamingo.load_state_dict(torch.load(checkpoint_path), strict=False)
model_flamingo.to(gpu_model_2) 

# 7min 

Using pad_token, but it is not set yet.


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards: 100%|██████████| 3/3 [00:31<00:00, 10.58s/it]


Flamingo model initialized with 1384781840 trainable parameters


Flamingo(
  (vision_encoder): VisionTransformer(
    (patchnorm_pre_ln): Identity()
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): Layer

## Choose specifc GPU for model

In [ ]:
# Define the GPU ID you want to use
gpu_id = 6

# Use the torch.cuda.device() context manager to set the current GPU
with torch.cuda.device(gpu_id):
    tokenizer = AutoTokenizer.from_pretrained(model_type+"/"+model_name, use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained(server_model_path+model_type+"/"+model_name, use_auth_token=True).to(torch.device("cuda"))

## Loading model

In [4]:
gpu_id = None
tokenizer = AutoTokenizer.from_pretrained(model_type+"/"+model_name, use_auth_token=True, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(server_model_path+model_type+"/"+model_name, use_auth_token=True, device_map="auto")

/mounts/work/pwicke/miniconda3/envs/schemas/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/mounts/work/pwicke/miniconda3/envs/schemas/lib/python3.8/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Vision-Language Experiment

* Using the exact same image that the participants see (with 4 options A,B,C,D) and the original prompt, the model only ever selects option A.
* Shufflinge the images, the model still selects mostly option A. 

This suggests that the choice is made based upon the name "A" and not the image itself. Hence, we encode the images with fake name-codes.

In [26]:
model_choices = dict()
arrows = ['UP', 'DOWN', 'LEFT', 'RIGHT']

for action_word in action_words:
    print(action_word)


    # Creating list of images for processing
    vision_x = []
    letters = ["KYMD", "PWTX", "YZPR", "DHNV"]
    l = list(range(1,5))
    random.shuffle(l)
    for idx, selected in enumerate(l):
        print("direction0"+str(selected)+" is "+letters[idx])
        vision_x.append(image_processor(Image.open("../../data/direction0"+str(i)+".png").convert("RGB")).unsqueeze(0))

    vision_x = torch.cat(vision_x, dim=0)
    vision_x = vision_x.unsqueeze(1).unsqueeze(0).to(gpu_model_2)
    tokenizer.padding_side = "left"


    prompt = "<image>Image KYMD<image>Image PWTX<image>Image YZPR<image>Image DHNV. Select the image that best depicts the event 'circle "+action_word+" square'. The best image is"

    lang_x = tokenizer(
        [prompt],
        return_tensors="pt",
    ).to(gpu_model_2)

    generated_text = model_flamingo.generate(
        vision_x=vision_x,
        lang_x=lang_x["input_ids"],
        attention_mask=lang_x["attention_mask"],
        max_new_tokens=50,
        do_sample=True,
        num_beams=3
    )

    print(tokenizer.decode(generated_text[0]))
    print()
    print()

Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


flew
direction02 is KYMD
direction01 is PWTX
direction03 is YZPR
direction04 is DHNV


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


<image>Image KYMD<image>Image PWTX<image>Image YZPR<image>Image DHNV. Select the image that best depicts the event 'circle flew square'. The best image is YZPR.<|endofchunk|>


hoped
direction04 is KYMD
direction01 is PWTX
direction02 is YZPR
direction03 is DHNV


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


...

<image>Image KYMD<image>Image PWTX<image>Image YZPR<image>Image DHNV. Select the image that best depicts the event 'circle hunted square'. The best image is KYMD.<|endofchunk|>


impacted
direction03 is KYMD
direction01 is PWTX
direction02 is YZPR
direction04 is DHNV


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


<image>Image KYMD<image>Image PWTX<image>Image YZPR<image>Image DHNV. Select the image that best depicts the event 'circle impacted square'. The best image is the first image.<|endofchunk|>


obeyed
direction01 is KYMD
direction03 is PWTX
direction02 is YZPR
direction04 is DHNV


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


<image>Image KYMD<image>Image PWTX<image>Image YZPR<image>Image DHNV. Select the image that best depicts the event 'circle obeyed square'. The best image is KYMD.<|endofchunk|>


argued with
direction04 is KYMD
direction01 is PWTX
direction03 is YZPR
direction02 is DHNV
<image>Image KYMD<image>Image PWTX<image>Image YZPR<image>Image DHNV. Select the image that best depicts the event 'circle argued with square'. The best image is the first image.<|endofchunk|>




## Run experiment with InstructBlip
We cannot use multiple images with InstructBlip


In [29]:
for action_word in action_words:
    print(action_word)
    image = Image.open("../../data/schema-choice.png").convert("RGB")
    prompt = "From the image, select the depiction (A, B, C or D) that best describes the event 'circle "+action_word+" square'. The best depiction is"
    inputs = processor_instructblip(images=image, text=prompt, return_tensors="pt").to(gpu_model_1)

    outputs = model_instructblip.generate(
    **inputs,
    do_sample=True,
    num_beams=5,
    max_length=256,
    min_length=1,
    top_p=0.9,
    repetition_penalty=1.5,
    length_penalty=1.0,
    temperature=1
    )
    generated_text = processor_instructblip.batch_decode(outputs, skip_special_tokens=True)[0].strip()

    print(generated_text)
    print()

flew
(b) square flew circle

hoped
(d) square hoped circle

pointed at
(d) circle pointed at square

warned
(d) square warned circle

rested
(d) square rested circle

smashed
(d) circle smashed square

lifted
(d) square lifted circle

tempted
(c) square tempted circle

rushed
(c) circle rushed square

fled
(d) square fled circle

increased
(d) circle increased square

offended
(d) circle offended square

floated
(d) square floated circle

sank
(d) square sank circle

gave to
(d) circle gave to square

owned
(c) circle owned square

showed
d

respected
(d) square respected circle

bombed
(d) square bombed circle

succeeded
(c) square succeeded circle

pulled
(d) square pulled circle

walked
(d) square walked circle

perched
(d) square perched circle

pushed
(d) square pushed circle

wanted
(d) square wanted circle

regretted
(d) square regretted circle

hunted
d

impacted
(c) square impacted circle

obeyed
(d) square obeyed circle

argued with
(c) square argued with circle



In [30]:
for action_word in action_words:
    print(action_word)
    image = Image.open("../../data/schema-choice.png").convert("RGB")
    prompt = "From the image, select the depiction (top-left, top-right, bottom-left or bottom-right) that best describes the event 'circle "+action_word+" square'. The best depiction is"
    inputs = processor_instructblip(images=image, text=prompt, return_tensors="pt").to(gpu_model_1)

    outputs = model_instructblip.generate(
    **inputs,
    do_sample=True,
    num_beams=5,
    max_length=256,
    min_length=1,
    top_p=0.9,
    repetition_penalty=1.5,
    length_penalty=1.0,
    temperature=1
    )
    generated_text = processor_instructblip.batch_decode(outputs, skip_special_tokens=True)[0].strip()

    print(generated_text)
    print()

flew
bottom-left

hoped
bottom-left

pointed at
bottom-left

warned
bottom-left

rested
bottom-left

smashed
bottom-left

lifted
bottom-left

tempted
bottom-left

rushed
bottom-left

fled
bottom-left

increased
bottom-left

offended
bottom-left

floated
bottom-left

sank
bottom-left

gave to
bottom-left

owned
bottom-left

showed
bottom-left

respected
bottom-left

bombed
bottom-left

succeeded
bottom-left

pulled
bottom-left

walked
bottom-left

perched
bottom-left

pushed
bottom-left

wanted
bottom-left

regretted
bottom-left

hunted
bottom-left

impacted
bottom-left

obeyed
bottom-left

argued with
bottom-left

